In [1]:
pip install backtrader


Note: you may need to restart the kernel to use updated packages.


In [16]:
import backtrader as bt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import math # For ceil in position sizing
import os # For checking file existence

In [17]:
# --- Data Loading and Preparation ---
def load_and_prepare_data(file_path='./XAU_15m_data.csv'):
    """Loads 15m data, prepares it, and resamples to daily."""
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['Date'])
    df = df.set_index('datetime')
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    df.ffill(inplace=True)
    df.dropna(inplace=True)

    df_daily = df.resample('D').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }).dropna()

    if df.empty:
        print(f"🛑 Error: 15-minute data loaded from '{file_path}' is empty after processing. Please check the data file.")
        exit()
    if df_daily.empty:
        print(f"🛑 Error: Daily data resampled from '{file_path}' is empty after processing. Please check the data file.")
        exit()

    return df, df_daily

In [23]:

# --- Configuration Parameters ---
# These can be adjusted for optimization
STRATEGY_PARAMS = {
    'fast_ema': 50,         # EMA period for daily trend (slower for less trades)
    'slow_ema': 100,        # EMA period for daily trend (slower for less trades)
    'atr_period': 14,       # ATR period for 15m volatility
    'volume_ma_period': 20, # Volume MA period for 15m confirmation
    'atr_multiplier_sl': 2.5, # Multiplier for ATR-based Stop Loss
    'risk_per_trade_percent': 0.01, # Risk 1% of portfolio per trade
    'trailing_stop_percent': 0.015, # 1.5% trailing stop for profit protection
    'rsi_period': 14,       # RSI period for 15m entry confirmation
    'rsi_oversold': 30,     # RSI oversold threshold
    'rsi_overbought': 70,   # RSI overbought threshold
    'stoch_rsi_period': 14, # Stochastic RSI period (for Highest/Lowest RSI)
    'stoch_rsi_k_period': 3, # Smoothing period for %K line of Stochastic RSI
    'stoch_rsi_d_period': 3, # Smoothing period for %D line of Stochastic RSI
    'stoch_rsi_oversold': 20, # Stochastic RSI oversold for sharper entries
    'stoch_rsi_overbought': 80,
    'macd_fast': 12,        # MACD fast EMA period for daily trend confirmation
    'macd_slow': 26,        # MACD slow EMA period for daily trend confirmation
    'macd_signal': 9,       # MACD signal EMA period
    'adx_period': 14,       # ADX period for daily trend strength
    'adx_min_strength': 25, # Minimum ADX for strong trend confirmation
    'volatility_threshold_atr_multiplier': 0.5, # Skip trades if current ATR is below this multiple of historical ATR
    'trading_months': list(range(1, 13)), # All months by default (1=Jan, 12=Dec)
    'fixed_commission': 0.0, # Fixed commission per trade
    'percent_commission': 0.0005, # 0.05% commission per trade
    'slippage_percent': 0.001, # 0.1% slippage
    'leverage': 5.0,        # Leverage for the account
    'initial_cash': 5000.0,
    'test_split_ratio': 0.2, # Ratio for train-test split (e.g., 0.2 means last 20% for testing)

    # --- Trendline Parameters (New from Pine Script) ---
    'enable_trendline_logic': True, # Set to False to disable this new logic
    'trendline_length': 14,         # Corresponds to Pine Script 'length' (Swing Detection Lookback)
    'slope_multiplier': 1.0,        # Corresponds to Pine Script 'mult' (Slope Multiplier)
    'slope_calc_method': 'Atr',     # 'Atr' or 'Stdev' for Slope Calculation Method
                                    # 'Linreg' from Pine Script is not implemented in this version due to complexity.
    'swing_lookback_period': 20,    # New: Lookback period for swing high/low detection
}

In [24]:
# --- Custom Trade Logger Analyzer ---
class TradeLogger(bt.Analyzer):
    """
    Custom analyzer to log detailed trade information.
    """
    def __init__(self):
        self.trades = []
        self.strat_instance = None # To get strategy parameters and indicator values

    def start(self):
        self.trades = []
        # Get a reference to the strategy instance
        self.strat_instance = self.strategy

    def notify_trade(self, trade):
        """Logs detailed information for closed trades."""
        if not trade.isclosed:
            return

        # Determine if long or short based on trade size
        is_long = trade.size > 0
        is_short = trade.size < 0

        # Get exit reason set by the strategy, or default to 'N/A'
        exit_reason = getattr(trade, 'exit_reason', 'N/A')

        # If the strategy didn't explicitly set an exit reason, infer one
        if exit_reason == 'N/A':
            if trade.pnlcomm > 0:
                exit_reason = 'Take Profit'
            elif trade.pnlcomm < 0:
                exit_reason = 'Stop Loss / Other Loss'
            else:
                exit_reason = 'Break Even / Other'

        log_entry = {
            'date': self.strat_instance.datas[0].datetime.date(0),
            'entry_date': trade.open_datetime().date(),
            'exit_date': trade.close_datetime().date(),
            'entry_price': trade.price,
            'exit_price': getattr(trade, 'priceclose', 'N/A'), # Use getattr for robustness
            'size': trade.size,
            'pnl': trade.pnl,
            'pnl_comm': trade.pnlcomm,
            'is_long': is_long, # Corrected: Use trade.size
            'is_short': is_short, # Corrected: Use trade.size
            # Access custom attributes set on the trade object
            'exit_reason': exit_reason, # Use the determined exit_reason
            'initial_stop_loss': getattr(trade, 'initial_stop_loss', 'N/A'),
            'take_profit': getattr(trade, 'take_profit', 'N/A'),
            'trailing_stop_final': getattr(trade, 'trailing_stop_final', 'N/A'),
            'current_ema_fast': float(self.strat_instance.ema_fast_daily[0]) if self.strat_instance.ema_fast_daily else 'N/A',
            'current_ema_slow': float(self.strat_instance.ema_slow_daily[0]) if self.strat_instance.ema_slow_daily else 'N/A',
            'current_rsi': float(self.strat_instance.rsi_15m[0]) if self.strat_instance.rsi_15m else 'N/A',
            'current_atr': float(self.strat_instance.atr_15m[0]) if self.strat_instance.atr_15m else 'N/A',
            'current_volume': float(self.strat_instance.data.volume[0]) if self.strat_instance.data.volume else 'N/A',
            'current_adx': float(self.strat_instance.adx_daily[0]) if self.strat_instance.adx_daily else 'N/A',
            # Removed 'current_macd_hist' as 'histo' is being removed
        }
        self.trades.append(log_entry)

    def get_analysis(self):
        return self.trades

In [25]:

# --- Advanced Strategy Design with Backtrader ---
class AdvancedStrategy(bt.Strategy):
    """
    A strategy combining multi-timeframe trend-following with mean-reversion principles,
    enhanced risk management, stricter entry, monthly trading filters,
    and now with dynamic trendline breakout detection.
    """
    params = STRATEGY_PARAMS

    # Declare all lines at the class level
    lines = ('trend_upper', 'trend_lower', 'trend_upos', 'trend_dnos', 'stoch_rsi_k_plot', 'stoch_rsi_d_plot',)

    def __init__(self):
        # Multi-timeframe data feeds: self.data is 15m, self.datas[1] is daily
        self.data_15m = self.datas[0]
        self.data_daily = self.datas[1]

        # Indicators for 15m data (signals)
        self.atr_15m = bt.indicators.AverageTrueRange(self.data_15m, period=self.p.atr_period)
        self.volume_ma_15m = bt.indicators.SimpleMovingAverage(self.data_15m.volume, period=self.p.volume_ma_period)
        self.rsi_15m = bt.indicators.RelativeStrengthIndex(self.data_15m.close, period=self.p.rsi_period)

        # --- Corrected Stochastic RSI Calculation ---
        # Calculate highest and lowest RSI over the stoch_rsi_period
        highest_rsi = bt.indicators.Highest(self.rsi_15m, period=self.p.stoch_rsi_period)
        lowest_rsi = bt.indicators.Lowest(self.rsi_15m, period=self.p.stoch_rsi_period)

        # Calculate the raw %K for Stochastic RSI
        # Add a small epsilon to the denominator to avoid division by zero if highest_rsi == lowest_rsi
        stoch_rsi_k_raw = 100 * ((self.rsi_15m - lowest_rsi) / (highest_rsi - lowest_rsi + 1e-9))

        # Smooth %K to get the final %K line (often called %K_StochRSI or just %K)
        self.stoch_rsi_k_line = bt.indicators.SimpleMovingAverage(stoch_rsi_k_raw, period=self.p.stoch_rsi_k_period)

        # Smooth %K again to get the %D line (often called %D_StochRSI or just %D)
        self.stoch_rsi_d_line = bt.indicators.SimpleMovingAverage(self.stoch_rsi_k_line, period=self.p.stoch_rsi_d_period)

        # Assign the calculated Stochastic RSI lines to the declared plot lines
        self.lines.stoch_rsi_k_plot = self.stoch_rsi_k_line
        self.lines.stoch_rsi_d_plot = self.stoch_rsi_d_line

        # Create a dummy object to mimic the structure of bt.indicators.Stochastic for backward compatibility
        # with how percK and percD were accessed in entry_logic.
        class StochasticRSIOutput:
            def __init__(self, k_line, d_line):
                self.percK = k_line
                self.percD = d_line
        self.stoch_rsi_15m = StochasticRSIOutput(self.stoch_rsi_k_line, self.stoch_rsi_d_line)


        # Indicators for continuous swing high/low detection on 15m data
        self.highest_low_15m = bt.indicators.Lowest(self.data_15m.low, period=self.p.swing_lookback_period)
        self.lowest_high_15m = bt.indicators.Highest(self.data_15m.high, period=self.p.swing_lookback_period)

        # Indicators for Daily data (trend)
        self.ema_fast_daily = bt.indicators.ExponentialMovingAverage(self.data_daily.close, period=self.p.fast_ema)
        self.ema_slow_daily = bt.indicators.ExponentialMovingAverage(self.data_daily.close, period=self.p.slow_ema)
        self.ema_crossover_daily = bt.indicators.CrossOver(self.ema_fast_daily, self.ema_slow_daily)
        self.adx_daily = bt.indicators.ADX(self.data_daily, period=self.p.adx_period)
        self.macd_daily = bt.indicators.MACD(
            self.data_daily.close,
            period_me1=self.p.macd_fast,
            period_me2=self.p.macd_slow,
            period_signal=self.p.macd_signal
        )

        # State tracking for existing strategy logic
        self.order = None
        self.trade_history = []
        self.swing_low = None
        self.swing_high = None
        self.entry_price = None
        self.trailing_stop_price = None
        self.initial_stop_loss_price = None
        self.take_profit_price = None
        self._pending_exit_reason = None # New attribute to store exit reason before notify_trade

        # To calculate historical ATR for volatility filter
        self.atr_history = []

        # --- Trendline specific initializations (New from Pine Script) ---
        if self.p.enable_trendline_logic:
            self.trendline_length = self.p.trendline_length
            self.slope_multiplier = self.p.slope_multiplier
            self.slope_calc_method = self.p.slope_calc_method

            # Indicators for slope calculation based on 15m data
            self.atr_15m_slope_calc = bt.indicators.AverageTrueRange(self.data_15m, period=self.trendline_length)
            self.stdev_15m_slope_calc = bt.indicators.StandardDeviation(self.data_15m.close, period=self.trendline_length)

            # Initialize internal values for trendlines
            self._upper_val = 0.0
            self._lower_val = 0.0
            self._slope_ph_val = 0.0
            self._slope_pl_val = 0.0
            self._upos_val = 0
            self._dnos_val = 0

            # Removed the incorrect bt.lines.C assignments here.
            # The lines are already defined in the 'lines' tuple at the class level.


    def notify_order(self, order):
        """Logs order status changes."""
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Size: {order.executed.size:.2f}')
                self.entry_price = order.executed.price
                # Initialize trailing stop after a buy, give some buffer
                self.trailing_stop_price = self.entry_price * (1 - self.p.trailing_stop_percent)
                # Removed direct access to self.position.trade here, as it's not always available immediately
                # The initial_stop_loss_price and take_profit_price are set in calculate_position_size_and_order
                # and then picked up by notify_trade.

            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, Profit: {order.executed.pnl:.2f}, Size: {order.executed.size:.2f}')
                self.entry_price = None  # Reset on sell
                self.trailing_stop_price = None  # Reset on sell
                self.initial_stop_loss_price = None
                self.take_profit_price = None
                self._pending_exit_reason = None # Reset pending exit reason on sell completion

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'Order {order.getstatusname()} - Price: {order.price:.2f}')

        self.order = None

    def notify_trade(self, trade):
        """Logs trade status changes and sets exit reason."""
        if not trade.isclosed:
            return

        # Set exit reason for logging
        if self._pending_exit_reason is not None:
            trade.exit_reason = self._pending_exit_reason
            self._pending_exit_reason = None # Reset after use
        elif trade.pnlcomm > 0:
            trade.exit_reason = 'Take Profit'  # Assuming positive PnL is TP
        elif trade.pnlcomm < 0:
            # Differentiate between fixed SL and trailing SL
            if self.trailing_stop_price is not None and trade.priceclose <= self.trailing_stop_price:
                trade.exit_reason = 'Trailing Stop Loss'
            elif self.initial_stop_loss_price is not None and trade.priceclose <= self.initial_stop_loss_price:
                trade.exit_reason = 'Fixed Stop Loss'
            else:
                trade.exit_reason = 'Trend Reversal / Other Loss'
        else:
            trade.exit_reason = 'Break Even / Other'

        # Store initial SL and TP on the trade object for logging
        # These are now set from the strategy-level attributes determined at entry time
        trade.initial_stop_loss = self.initial_stop_loss_price
        trade.take_profit = self.take_profit_price
        trade.trailing_stop_final = self.trailing_stop_price  # Capture final trailing stop value

        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}, Reason: {trade.exit_reason}')

    def log(self, txt, dt=None):
        """Custom logging function."""
        dt = dt or self.data_15m.datetime.datetime(0)  # Use 15m datetime for logging
        print(f'{dt.isoformat()} - {txt}')

    def prenext(self):
        """Called before next() when all data feeds are not yet synchronized."""
        # Accumulate ATR history for volatility filter
        if len(self.atr_15m) > 0:
            self.atr_history.append(self.atr_15m[0])

    def _calculate_slope(self, data_feed):
        """
        Calculates slope based on the selected method ('Atr' or 'Stdev').
        Mimics Pine Script's slope calculation.
        """
        # Ensure enough data for calculation
        if len(data_feed) < self.trendline_length:
            return 0.0

        if self.slope_calc_method == 'Atr':
            # ATR is already calculated as self.atr_15m_slope_calc
            return (self.atr_15m_slope_calc[0] / self.trendline_length) * self.slope_multiplier
        elif self.slope_calc_method == 'Stdev':
            # Stdev is already calculated as self.stdev_15m_slope_calc
            return (self.stdev_15m_slope_calc[0] / self.trendline_length) * self.slope_multiplier
        # 'Linreg' from Pine Script is not implemented in this version.
        return 0.0

    def next(self):
        """Main strategy logic executed on each 15m bar."""

        # --- Robust Indicator Warm-up Check ---
        # Ensure enough bars have passed for all indicators to be calculated
        # For 15m data: ATR, RSI, Stoch RSI, Volume MA, Trendline pivots
        # The Stochastic RSI calculation now depends on RSI period + Stoch RSI period + K period + D period
        min_stoch_rsi_bars = (self.p.rsi_period +
                              self.p.stoch_rsi_period +
                              self.p.stoch_rsi_k_period +
                              self.p.stoch_rsi_d_period)

        min_15m_bars_needed = max(self.p.atr_period,
                                  self.p.volume_ma_period,
                                  self.p.swing_lookback_period, # For highest_low/lowest_high
                                  self.p.trendline_length + 1, # For trendline pivots
                                  min_stoch_rsi_bars)

        # For daily data: EMAs, ADX, MACD (histo needs slow + signal periods)
        min_daily_bars_needed = max(self.p.slow_ema,
                                    self.p.adx_period,
                                    self.p.macd_slow + self.p.macd_signal)


        # Check if enough data is available for all indicators on both timeframes
        if (len(self.data_15m) < min_15m_bars_needed or
            len(self.data_daily) < min_daily_bars_needed):
            # self.log(f'Not enough data yet for all indicators to warm up. 15m bars: {len(self.data_15m)}/{min_15m_bars_needed}, Daily bars: {len(self.data_daily)}/{min_daily_bars_needed}')
            return # Not enough data yet for all indicators to warm up

        if self.order:
            return  # An order is pending, nothing to do

        # Ensure daily data is available and synchronized for trend indicators
        # In Backtrader, `next` is is called when all data feeds have new bars.
        # So, `self.datas[1].close[0]` refers to the current daily close.
        if not self.data_daily.datetime.date(0) == self.data_15m.datetime.date(0):
            # This condition might be true if daily data is not aligned perfectly
            # or if the 15m bar is the first for a new day.
            # For backtrader, `next` ensures data is ready, so this check is mostly for conceptual alignment.
            pass

        # Update swing high/low based on the new indicators
        # Ensure enough data for swing lookback period
        if len(self.highest_low_15m) >= self.p.swing_lookback_period and \
           len(self.lowest_high_15m) >= self.p.swing_lookback_period:
            self.swing_low = self.highest_low_15m[0]
            self.swing_high = self.lowest_high_15m[0]
        else:
            self.swing_low = None
            self.swing_high = None

        # Check trading months
        current_month = self.data_15m.datetime.date(0).month
        if current_month not in self.p.trading_months:
            return  # Do not trade in non-trading months

        # --- Volatility Filter ---
        if len(self.atr_history) > self.p.atr_period * 2:  # Ensure enough history
            avg_atr = np.mean(self.atr_history[-self.p.atr_period * 2:])
            if self.atr_15m[0] < avg_atr * self.p.volatility_threshold_atr_multiplier:
                # self.log(f'Skipping trade due to low volatility (ATR: {self.atr_15m[0]:.2f})')
                return

        # --- Trendline and Breakout Logic (New from Pine Script) ---
        if self.p.enable_trendline_logic:
            # Ensure enough data for lookback period for pivot detection and indicator calculations
            # We need at least 'trendline_length' bars to look back for pivots.
            if len(self.data_15m) < self.trendline_length + 1 or \
               len(self.atr_15m_slope_calc) < self.trendline_length + 1 or \
               len(self.stdev_15m_slope_calc) < self.trendline_length + 1:
                return

            current_close = self.data_15m.close[0]
            current_high = self.data_15m.high[0]
            current_low = self.data_15m.low[0]

            # Simplified Pivot Detection: Check if current high/low is the highest/lowest in the past 'length' bars
            # This is a basic implementation of pivot detection. Pine Script's pivothigh/pivotlow
            # can be more complex, potentially looking ahead or using different criteria.
            is_ph = True # Assume current bar is a pivot high initially
            for i in range(1, self.trendline_length + 1): # Look back 'trendline_length' bars
                if self.data_15m.high[-i] >= current_high: # If any past high is greater or equal
                    is_ph = False # It's not a pivot high
                    break

            is_pl = True # Assume current bar is a pivot low initially
            for i in range(1, self.trendline_length + 1): # Look back 'trendline_length' bars
                if self.data_15m.low[-i] <= current_low: # If any past low is less or equal
                    is_pl = False # It's not a pivot low
                    break

            # Calculate current slope based on chosen method
            current_slope = self._calculate_slope(self.data_15m)

            # Update persistent slope values only when a new pivot is detected
            if is_ph:
                self._slope_ph_val = current_slope
            if is_pl:
                self._slope_pl_val = current_slope

            # Store previous breakout values to detect state changes (e.g., 0 -> 1)
            prev_upos_val = self._upos_val
            prev_dnos_val = self._dnos_val

            # Update Trendline Values (upper and lower)
            # If a new pivot high is found, the upper trendline starts at that pivot.
            # Otherwise, it extends downwards using the last pivot high's slope.
            if is_ph:
                self._upper_val = current_high
            else:
                self._upper_val -= self._slope_ph_val

            # If a new pivot low is found, the lower trendline starts at that pivot.
            # Otherwise, it extends upwards using the last pivot low's slope.
            if is_pl:
                self._lower_val = current_low
            else:
                self._lower_val += self._slope_pl_val

            # Update Backtrader lines with the current trendline values
            self.lines.trend_upper[0] = self._upper_val
            self.lines.trend_lower[0] = self._lower_val

            # Breakout Detection (upos, dnos)
            # Reset breakout flag if a new pivot is formed.
            # Otherwise, check if the current close price crosses the trendline.
            if is_ph:
                self._upos_val = 0  # Reset on new pivot high
            elif current_close > self._upper_val:  # Upward breakout if close crosses above upper trendline
                self._upos_val = 1

            if is_pl:
                self._dnos_val = 0  # Reset on new pivot low
            elif current_close < self._lower_val:  # Downward breakout if close crosses below lower trendline
                self._dnos_val = 1

            # Update Backtrader lines with the current breakout flags
            self.lines.trend_upos[0] = self._upos_val
            self.lines.trend_dnos[0] = self._dnos_val

            # --- Integrate Trendline Breakouts into Trading Logic ---
            # For a long-only strategy:
            # - An upward breakout (`upos`) can be an additional entry confirmation.
            # - A downward breakout (`dnos`) can be an early exit signal.

            if not self.position:  # Not in the market
                # Example: Add upward breakout as an entry condition.
                # You can add `and self.lines.trend_upos[0] == 1` to your `entry_logic` conditions.
                if self._upos_val > prev_upos_val: # Detects when upos changes from 0 to 1 (a new breakout event)
                    self.log(f'TRENDLINE: Upward Breakout Detected at {self.data_15m.datetime.datetime(0).isoformat()} - Price: {current_close:.2f}')

            else:  # In a long position
                # Example: Use downward breakout as an additional exit condition.
                if self._dnos_val > prev_dnos_val: # Detects when dnos changes from 0 to 1 (a new breakout event)
                    self.log(f'TRENDLINE: Downward Breakout Detected, Closing Position at {self.data_15m.datetime.datetime(0).isoformat()} - Price: {current_close:.2f}')
                    self._pending_exit_reason = 'Downward Trendline Breakout' # Set pending reason
                    self.close()
                    return  # Exit next() after closing position to avoid other logic in the same bar

        # --- Trend and Swing Point Logic (based on 15m data for swing points, daily for trend) ---
        # This part remains from your original strategy
        if self.ema_crossover_daily[0] > 0: # Bullish crossover on daily
            if self.data_15m.close[0] > self.ema_fast_daily[0] and self.data_15m.close[0] > self.ema_slow_daily[0]:
                if self.swing_low is None:
                    self.swing_low = self.data_15m.low[0]
                    self.swing_high = self.data_15m.high[0]
                else:
                    self.swing_low = min(self.swing_low, self.data_15m.low[0])
                    self.swing_high = max(self.swing_high, self.data_15m.high[0])
        elif self.ema_crossover_daily[0] < 0: # Bearish crossover on daily
            self.swing_low = None
            self.swing_high = None

        # --- Entry Logic (Long only) ---
        if not self.position:  # Not in the market
            self.entry_logic()
        else: # In the market (Long position)
            self.exit_logic()

    def entry_logic(self):
        """Handles conditions for opening a long position."""
        # Ensure all indicators have enough data points
        if (len(self.ema_fast_daily) < self.p.slow_ema or
            len(self.adx_daily) < self.p.adx_period or
            len(self.rsi_15m) < self.p.rsi_period or
            len(self.stoch_rsi_d_line) < 1 or # Check if stoch_rsi_d_line has at least one value
            len(self.atr_15m) < self.p.atr_period or
            len(self.volume_ma_15m) < self.p.volume_ma_period):
            return

        # 1. Daily Trend Filter: Bullish EMA Crossover & ADX confirms trend strength
        if not (self.ema_crossover_daily[0] > 0 and self.adx_daily[0] > self.p.adx_min_strength):
            return

        # 2. Daily MACD Confirmation: MACD line above Signal line
        if not (self.macd_daily.macd[0] > self.macd_daily.signal[0]):
            return

        # 3. 15m Pullback to Fibonacci level
        if self.swing_low is None or self.swing_high is None:
            return

        swing_range = self.swing_high - self.swing_low
        if swing_range <= 0:
            return

        fib_618_level = self.swing_high - (swing_range * 0.618)

        # Condition: Price pulls back to the 61.8% Fibonacci level and closes above it
        if not (self.data_15m.low[0] <= fib_618_level and self.data_15m.close[0] > fib_618_level):
            return

        # 4. 15m Volume Confirmation: Volume is above its moving average
        if not (self.data_15m.volume[0] > self.volume_ma_15m[0]):
            return

        # 5. 15m RSI & Stochastic RSI Confirmation for sharper pullback entries
        # Accessing percK and percD directly from the dummy object
        if not (self.rsi_15m[0] < self.p.rsi_overbought and
                self.stoch_rsi_15m.percK[0] < self.p.stoch_rsi_overbought and
                self.stoch_rsi_15m.percK[0] > self.p.stoch_rsi_oversold and
                self.stoch_rsi_15m.percK[0] > self.stoch_rsi_15m.percD[0]):
            return

        # --- NEW: Optional Trendline Breakout Confirmation for Entry ---
        # Add this condition if you want the upward trendline breakout to be a mandatory entry signal.
        # if self.p.enable_trendline_logic and not (self.lines.trend_upos[0] == 1):
        #     return

        # All conditions met, calculate position size and place order
        self.calculate_position_size_and_order()

    def calculate_position_size_and_order(self):
        """Calculates position size based on ATR and places a buy order."""
        current_atr = self.atr_15m[0]
        if current_atr <= 0:
            self.log('ATR is zero or negative, cannot calculate position size.')
            return

        # Risk fixed percentage of account equity
        account_value = self.broker.getvalue()
        risk_amount = account_value * self.p.risk_per_trade_percent

        initial_sl_price = self.swing_low - (current_atr * self.p.atr_multiplier_sl)
        take_profit_price_calc = self.swing_high + ((self.swing_high - self.swing_low) * 0.618) # Fibonacci extension TP

        if initial_sl_price >= self.data_15m.close[0]:
            self.log('Calculated initial stop loss is above or at current price, cannot place order.')
            return

        risk_per_share = self.data_15m.close[0] - initial_sl_price
        if risk_per_share <= 0:
            self.log('Risk per share is zero or negative, cannot calculate position size.')
            return

        size = risk_amount / risk_per_share
        size = math.floor(size)

        if size > 0:
            self.log(f'BUY CREATE at {self.data_15m.close[0]:.2f} with size {size:.2f}')
            self.initial_stop_loss_price = initial_sl_price # Store on strategy instance
            self.take_profit_price = take_profit_price_calc # Store on strategy instance
            self.order = self.buy(size=size)
        else:
            self.log('Calculated position size is zero, not placing order.')

    def exit_logic(self):
        """Handles conditions for closing a long position."""
        # Update trailing stop price
        if self.entry_price is not None and self.trailing_stop_price is not None:
            self.trailing_stop_price = max(self.trailing_stop_price, self.data_15m.close[0] * (1 - self.p.trailing_stop_percent))

        # 1. Trend reversal (Daily EMA death cross) - Primary exit for trend
        if self.ema_crossover_daily[0] < 0:
            self.log(f'SELL CREATE (Trend Reversal) at {self.data_15m.close[0]:.2f}')
            self._pending_exit_reason = 'Trend Reversal' # Set pending reason
            self.close()
            return

        # 2. Trailing Stop Loss
        if self.data_15m.close[0] <= self.trailing_stop_price:
            self.log(f'SELL CREATE (Trailing Stop Loss) at {self.trailing_stop_price:.2f}')
            self._pending_exit_reason = 'Trailing Stop Loss' # Set pending reason
            self.close()
            return

        # 3. Fixed Stop Loss (ATR-based, from initial swing low)
        if self.swing_low is not None:
            sl_price = self.swing_low - (self.atr_15m[0] * self.p.atr_multiplier_sl)
            if self.data_15m.low[0] <= sl_price:
                self.log(f'SELL CREATE (Fixed Stop Loss) at {sl_price:.2f}')
                self._pending_exit_reason = 'Fixed Stop Loss' # Set pending reason
                self.close()
                return

        # 4. Take Profit (Fibonacci extension)
        if self.swing_high is not None and self.swing_low is not None:
            swing_range = self.swing_high - self.swing_low
            if swing_range > 0:
                tp_price = self.swing_high + (swing_range * 0.618)
                if self.data_15m.high[0] >= tp_price:
                    self.log(f'SELL CREATE (Take Profit) at {tp_price:.2f}')
                    self._pending_exit_reason = 'Take Profit' # Set pending reason
                    self.close()
                    return

In [26]:
# --- Backtesting and Optimization Framework ---
def run_backtest(data_15m, data_daily, params, is_optimization=False):
    """Runs a single backtest with given parameters."""
    cerebro = bt.Cerebro()
    cerebro.addstrategy(AdvancedStrategy, **params)

    # Add data feeds
    cerebro.adddata(bt.feeds.PandasData(dataname=data_15m, timeframe=bt.TimeFrame.Minutes, compression=15))
    cerebro.adddata(bt.feeds.PandasData(dataname=data_daily, timeframe=bt.TimeFrame.Days, compression=1))

    # Broker settings
    cerebro.broker.set_cash(params['initial_cash'])
    cerebro.broker.setcommission(commission=params['percent_commission'],
                                 commtype=bt.CommInfoBase.COMM_PERC,
                                 mult=1.0, # For forex, 1.0 means actual price
                                 leverage=params['leverage'])
    cerebro.broker.set_slippage_perc(params['slippage_percent'])
    cerebro.broker.set_fundmode(True, fundstartval=params['initial_cash']) # Enable fund mode for leverage

    # Add Analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio', timeframe=bt.TimeFrame.Days, compression=252)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

    # Dummy classes for analyzers that might not be present in all Backtrader versions
    class DummyParams:
        def _getkwargs(self):
            return {}

    class DummyCalmar:
        def __init__(self):
            self.p = DummyParams()
        def get_analysis(self): return {'calmarratio': 'N/A'}
        def _start(self): pass
        def _notify_cashvalue(self, cash, value): pass
        def _notify_fund(self, cash, value, fundvalue, fundshares): pass
        def _prenext(self): pass
        def _nextstart(self): pass
        def _next(self): pass
        def _stop(self): pass
        def _notify_order(self, order): pass
        def _notify_trade(self, trade): pass

    class DummySortino:
        def __init__(self):
            self.p = DummyParams()
        def get_analysis(self): return {'sortinoratio': 'N/A'}
        def _start(self): pass
        def _notify_cashvalue(self, cash, value): pass
        def _notify_fund(self, cash, value, fundvalue, fundshares): pass
        def _prenext(self): pass
        def _nextstart(self): pass
        def _next(self): pass
        def _stop(self): pass
        def _notify_order(self, order): pass
        def _notify_trade(self, trade): pass

    if hasattr(bt.analyzers, 'CalmarRatio'):
        cerebro.addanalyzer(bt.analyzers.CalmarRatio, _name='calmar_ratio', timeframe=bt.TimeFrame.Days, compression=252)
    else:
        print("Warning: CalmarRatio analyzer not found. Skipping.")
        cerebro.addanalyzer(DummyCalmar, _name='calmar_ratio')

    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Months) # For monthly returns
    cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn') # System Quality Number
    cerebro.addanalyzer(TradeLogger, _name='trade_logger') # Custom trade logger

    if hasattr(bt.analyzers, 'SortinoRatio'):
        cerebro.addanalyzer(bt.analyzers.SortinoRatio, _name='sortino_ratio', timeframe=bt.TimeFrame.Days, compression=252)
    else:
        print("Warning: SortinoRatio analyzer not found. Skipping.")
        cerebro.addanalyzer(DummySortino, _name='sortino_ratio')


    # Run backtest
    results = cerebro.run()
    strat = results[0]

    # Extract analysis
    final_value = cerebro.broker.getvalue()
    profit = final_value - params['initial_cash']
    profit_pct = (profit / params['initial_cash']) * 100

    analysis = {
        'sharpe': strat.analyzers.sharpe_ratio.get_analysis(),
        'drawdown': strat.analyzers.drawdown.get_analysis(),
        'returns': strat.analyzers.returns.get_analysis(),
        'trades': strat.analyzers.trade_analyzer.get_analysis(),
        'sortino': strat.analyzers.sortino_ratio.get_analysis(), # This might be 'N/A' if the analyzer was skipped
        'calmar': strat.analyzers.calmar_ratio.get_analysis(),
        'time_return': strat.analyzers.time_return.get_analysis(),
        'sqn': strat.analyzers.sqn.get_analysis(),
        'trade_log': strat.analyzers.trade_logger.get_analysis()
    }

    # Calculate additional metrics
    total_return = (final_value / params['initial_cash']) - 1.0
    cagr = analysis['returns'].get('rnorm100', 0.0)

    max_drawdown = analysis['drawdown'].get('max', {}).get('drawdown', 0.0)

    # Trade statistics
    won_trades_total = analysis['trades'].get('won', {}).get('total', 0)
    lost_trades_total = analysis['trades'].get('lost', {}).get('total', 0)
    total_trades = analysis['trades'].get('total', {}).get('total', 1)

    win_rate = won_trades_total / total_trades if total_trades > 0 else 0.0

    won_pnl_total = analysis['trades'].get('won', {}).get('pnl', {}).get('total', 0)
    lost_pnl_total = analysis['trades'].get('lost', {}).get('pnl', {}).get('total', 1)

    profit_factor = abs(won_pnl_total) / abs(lost_pnl_total) if abs(lost_pnl_total) > 0 else float('inf')

    avg_win_pnl = analysis['trades'].get('won', {}).get('pnl', {}).get('average', 0)
    avg_loss_pnl = analysis['trades'].get('lost', {}).get('pnl', {}).get('average', 0)

    expectancy = (win_rate * avg_win_pnl) + ((1 - win_rate) * avg_loss_pnl) # Simplified expectancy calculation

    # Rolling Sharpe Ratio (requires daily returns)
    daily_returns_series = pd.Series(analysis['returns'].get('rnorm'), index=data_daily.index)
    rolling_sharpe = daily_returns_series.rolling(window=252).std() # Annualized std dev of daily returns
    if not rolling_sharpe.empty:
        # Assuming risk-free rate is 0 for simplicity in this example
        rolling_sharpe = (daily_returns_series.rolling(window=252).mean() / rolling_sharpe) * np.sqrt(252)
    else:
        rolling_sharpe = pd.Series() # Empty series if no data

    metrics = {
        'initial_capital': params['initial_cash'],
        'final_value': final_value,
        'net_profit': profit,
        'net_profit_pct': profit_pct,
        'total_return': total_return,
        'cagr': cagr,
        'sharpe_ratio': analysis['sharpe'].get('sharperatio', 'N/A'),
        'sortino_ratio': analysis['sortino'].get('sortinoratio', 'N/A'),
        'calmar_ratio': analysis['calmar'].get('calmarratio', 'N/A'),
        'max_drawdown': max_drawdown,
        'win_rate': win_rate,
        'profit_factor': profit_factor,
        'total_trades': total_trades,
        'avg_trade_pnl': analysis['trades'].get('pnl', {}).get('average', 0),
        'expectancy': expectancy,
        'sqn': analysis['sqn'].get('sqn', 'N/A'),
        'monthly_returns': analysis['time_return'],
        'rolling_sharpe': rolling_sharpe,
        'trade_log': analysis['trade_log']
    }

    if not is_optimization:
        # Generate detailed report and visualization only for main backtest
        generate_strategy_report(metrics, params)
        generate_plotly_visualization(data_15m, data_daily, strat, metrics)
        save_trade_log_to_csv(metrics['trade_log'])

    return metrics, strat

def generate_strategy_report(metrics, params):
    """Generates a detailed strategy performance report."""
    report_content = f"""
--- XAUUSD ADVANCED STRATEGY BACKTEST REPORT ---
Date Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

--- Strategy Parameters ---
Fast EMA Period: {params['fast_ema']}
Slow EMA Period: {params['slow_ema']}
ATR Period: {params['atr_period']}
Volume MA Period: {params['volume_ma_period']}
ATR Multiplier SL: {params['atr_multiplier_sl']}
Risk Per Trade Percent: {params['risk_per_trade_percent']:.2%}
Trailing Stop Percent: {params['trailing_stop_percent']:.2%}
RSI Period: {params['rsi_period']}
Stoch RSI Period: {params['stoch_rsi_period']}
MACD Fast/Slow/Signal: {params['macd_fast']}/{params['macd_slow']}/{params['macd_signal']}
ADX Period: {params['adx_period']}
ADX Min Strength: {params['adx_min_strength']}
Volatility Threshold (ATR Multiplier): {params['volatility_threshold_atr_multiplier']}
Trading Months: {', '.join(map(str, params['trading_months'])) if params['trading_months'] else 'All'}
Commission (Fixed): {params['fixed_commission']}
Commission (Percent): {params['percent_commission']:.4%}
Slippage (Percent): {params['slippage_percent']:.2%}
Leverage: {params['leverage']}x

# --- NEW: Trendline Parameters ---
Enable Trendline Logic: {params['enable_trendline_logic']}
Trendline Lookback Length: {params['trendline_length']}
Slope Multiplier: {params['slope_multiplier']}
Slope Calculation Method: {params['slope_calc_method']}


--- Overall Performance ---
Initial Capital: ₹{metrics['initial_capital']:.2f}
Final Value: ₹{metrics['final_value']:.2f}
Net Profit: ₹{metrics['net_profit']:.2f} ({metrics['net_profit_pct']:.2f}%)
Total Return: {metrics['total_return']:.2%}
CAGR (Compound Annual Growth Rate): {metrics['cagr']:.2f}%

--- Risk & Drawdown ---
Max Drawdown: {metrics['max_drawdown']:.2f}%
Sharpe Ratio: {metrics['sharpe_ratio'] if metrics['sharpe_ratio'] != 'N/A' else 'N/A'}
Sortino Ratio: {metrics['sortino_ratio'] if metrics['sortino_ratio'] != 'N/A' else 'N/A'}
Calmar Ratio: {metrics['calmar_ratio'] if metrics['calmar_ratio'] != 'N/A' else 'N/A'}
SQN (System Quality Number): {metrics['sqn'] if metrics['sqn'] != 'N/A' else 'N/A'}

--- Trade Statistics ---
Total Trades: {metrics['total_trades']}
Win Rate: {metrics['win_rate']:.2%}
Profit Factor: {metrics['profit_factor']:.2f}
Average Trade PnL: ₹{metrics['avg_trade_pnl']:.2f}
Expectancy: ₹{metrics['expectancy']:.2f}

--- Monthly Returns ---
"""
    for date_str, ret in metrics['monthly_returns'].items():
        report_content += f"{date_str}: {ret:.2%}\n"

    report_file_path = "strategy_report.txt"
    with open(report_file_path, "w", encoding='utf-8') as f:
        f.write(report_content)
    print(f"✅ Strategy report saved to '{report_file_path}'")

def save_trade_log_to_csv(trade_log):
    """Saves the detailed trade log to a CSV file."""
    if trade_log:
        trade_df = pd.DataFrame(trade_log)
        trade_df.to_csv('trades_log.csv', index=False)
        print("✅ Detailed trade log saved to 'trades_log.csv'")
    else:
        print("No trades to log.")

def generate_plotly_visualization(data_15m, data_daily, strat, metrics):
    """Generates a comprehensive Plotly visualization."""
    df_plot = data_15m.copy() # Use the 15m dataframe for plotting candles
    dates = df_plot.index

    # If dates is empty, there's no data to plot, so return early.
    if dates.empty:
        print("🛑 No 15-minute data available for plotting. Skipping visualization.")
        return

    # Get EMA values from the strategy's lines (daily)
    # Ensure EMA series have the correct index from the original df_daily
    # Corrected: Access EMA values using .lines.ema.array attribute
    ema_fast_values = list(strat.ema_fast_daily.lines.ema.array)
    ema_slow_values = list(strat.ema_slow_daily.lines.ema.array)

    # Create pandas Series with correct daily index
    # Ensure length match by slicing the values array if it's longer than the index
    fast_ema_daily_plot = pd.Series(ema_fast_values[-len(data_daily.index):], index=data_daily.index)
    slow_ema_daily_plot = pd.Series(ema_slow_values[-len(data_daily.index):], index=data_daily.index)


    # Reindex daily EMAs to 15m dates for plotting continuity (forward-fill)
    fast_ema_plot = fast_ema_daily_plot.reindex(dates, method='ffill')
    slow_ema_plot = slow_ema_daily_plot.reindex(dates, method='ffill')

    # Prepare portfolio equity and drawdown
    initial_cash = metrics['initial_capital']
    daily_returns_series = pd.Series(strat.analyzers.returns.get_analysis().get('rnorm'), index=data_daily.index)
    equity_curve = (1 + daily_returns_series).cumprod() * initial_cash
    drawdown_series = pd.Series(strat.analyzers.drawdown.get_analysis().get('drawdown'), index=data_daily.index)

    # Reindex equity_curve and drawdown_series to the full 15m dates for plotting continuity
    equity_curve = equity_curve.reindex(dates, method='ffill')
    drawdown_series = drawdown_series.reindex(dates, method='ffill')

    # Buy/Sell signals from strategy
    signals_df = pd.DataFrame(metrics['trade_log'])
    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    if not signals_df.empty:
        signals_df['date'] = pd.to_datetime(signals_df['date'])
        signals_df.set_index('date', inplace=True)
        # Filter for buy and sell signals based on 'is_long' and 'is_short' or 'exit_reason'
        buy_signals = signals_df[signals_df['is_long'] == True]
        sell_signals = signals_df[signals_df['is_short'] == True]


    # Plotly Subplots Setup
    fig = make_subplots(
        rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        subplot_titles=(
            'Price Chart with Signals & Daily EMAs',
            'Portfolio Equity Curve & Drawdown',
            'Monthly Returns',
            'Trade PnL Histogram'
        ),
        row_heights=[0.5, 0.2, 0.15, 0.15]
    )

    # --- Plot 1: Price, EMAs, Buy/Sell ---
    fig.add_trace(go.Candlestick(
        x=dates,
        open=df_plot['Open'],
        high=df_plot['High'],
        low=df_plot['Low'],
        close=df_plot['Close'],
        name='XAUUSD'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=fast_ema_plot.index,
        y=fast_ema_plot,
        line=dict(color='cyan', width=1),
        name=f'Daily Fast EMA ({STRATEGY_PARAMS["fast_ema"]})'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=slow_ema_plot.index,
        y=slow_ema_plot,
        line=dict(color='orange', width=1),
        name=f'Daily Slow EMA ({STRATEGY_PARAMS["slow_ema"]})'
    ), row=1, col=1)

    # Add buy signals
    if not buy_signals.empty:
        fig.add_trace(go.Scatter(
            x=buy_signals.index,
            y=buy_signals['entry_price'], # Use entry_price for buy signals
            mode='markers',
            marker=dict(color='green', size=10, symbol='triangle-up'),
            name='Buy Signal'
        ), row=1, col=1)

    # Add sell signals with exit reason annotation
    if not sell_signals.empty:
        for i, row in sell_signals.iterrows():
            fig.add_trace(go.Scatter(
                x=[i],
                y=[row['exit_price']], # Use exit_price for sell signals
                mode='markers',
                marker=dict(color='red', size=10, symbol='triangle-down'),
                name=f'Sell Signal ({row["exit_reason"]})' if 'exit_reason' in row else 'Sell Signal'
            ), row=1, col=1)
            if 'exit_reason' in row:
                fig.add_annotation(
                    x=i,
                    y=row['exit_price'],
                    text=row['exit_reason'],
                    showarrow=True,
                    arrowhead=0,
                    yshift=10,
                    font=dict(size=8, color="red"),
                    row=1, col=1
                )

    # --- NEW: Plot Trendlines if enabled ---
    if STRATEGY_PARAMS['enable_trendline_logic']:
        # Reindex strategy trendline outputs to 15m dates for plotting continuity
        # Corrected: Access trendline values using .array attribute from strat.lines
        if hasattr(strat.lines, 'trend_upper') and len(strat.lines.trend_upper.array) > 0:
            # Ensure length match by slicing the values array if it's longer than the index
            trend_upper_values = strat.lines.trend_upper.array[-len(dates):]
            trend_upper_plot = pd.Series(trend_upper_values, index=dates)
            fig.add_trace(go.Scatter(
                x=trend_upper_plot.index,
                y=trend_upper_plot,
                line=dict(color='teal', width=1, dash='dash'),
                name='Upper Trendline'
            ), row=1, col=1)
        else:
            print("Warning: Upper Trendline plot skipped due to insufficient data or missing line.")

        if hasattr(strat.lines, 'trend_lower') and len(strat.lines.trend_lower.array) > 0:
            # Ensure length match by slicing the values array if it's longer than the index
            trend_lower_values = strat.lines.trend_lower.array[-len(dates):]
            trend_lower_plot = pd.Series(trend_lower_values, index=dates)
            fig.add_trace(go.Scatter(
                x=trend_lower_plot.index,
                y=trend_lower_plot,
                line=dict(color='red', width=1, dash='dash'),
                name='Lower Trendline'
            ), row=1, col=1)
        else:
            print("Warning: Lower Trendline plot skipped due to insufficient data or missing line.")


    # --- Plot Stochastic RSI if enabled ---
    if hasattr(strat.lines, 'stoch_rsi_k_plot') and hasattr(strat.lines, 'stoch_rsi_d_plot'):
        # Ensure there's enough data in the lines before trying to plot
        if len(strat.lines.stoch_rsi_k_plot.array) > 0 and len(strat.lines.stoch_rsi_d_plot.array) > 0:
            # Ensure length match by slicing the values array if it's longer than the index
            stoch_rsi_k_values = strat.lines.stoch_rsi_k_plot.array[-len(dates):]
            stoch_rsi_d_values = strat.lines.stoch_rsi_d_plot.array[-len(dates):]

            stoch_rsi_k_plot = pd.Series(stoch_rsi_k_values, index=dates)
            stoch_rsi_d_plot = pd.Series(stoch_rsi_d_values, index=dates)

            # Add a new subplot for Stochastic RSI or overlay on an existing one
            # For now, let's add it to a new row for clarity, or it can be combined with RSI
            # For simplicity, I'll add it to the main price chart for now, but in a real scenario,
            # you might want a separate subplot for oscillators.
            # Adding to row 1 for now, but consider a separate subplot if it clutters.
            fig.add_trace(go.Scatter(
                x=stoch_rsi_k_plot.index,
                y=stoch_rsi_k_plot,
                line=dict(color='purple', width=1),
                name='Stoch RSI %K'
            ), row=1, col=1)

            fig.add_trace(go.Scatter(
                x=stoch_rsi_d_plot.index,
                y=stoch_rsi_d_plot,
                line=dict(color='magenta', width=1, dash='dot'),
                name='Stoch RSI %D'
            ), row=1, col=1)
        else:
            print("Warning: Stochastic RSI plots skipped due to insufficient data for Stochastic RSI.")


    # --- Plot 2: Equity + Drawdown ---
    fig.add_trace(go.Scatter(
        x=equity_curve.index,
        y=equity_curve.values,
        line=dict(color='blue', width=2),
        name='Equity Curve'
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=drawdown_series.index,
        y=-drawdown_series.values, # Drawdown is typically shown as negative or positive percentage
        fill='tozeroy',
        line=dict(color='rgba(255,0,0,0.3)', width=1),
        name='Drawdown'
    ), row=2, col=1)

    # --- Plot 3: Monthly Returns Bar Chart ---
    monthly_returns_df = pd.Series(metrics['monthly_returns']).to_frame(name='Return')
    monthly_returns_df.index = pd.to_datetime(monthly_returns_df.index)
    monthly_returns_df['Month'] = monthly_returns_df.index.strftime('%Y-%m')

    fig.add_trace(go.Bar(
        x=monthly_returns_df['Month'],
        y=monthly_returns_df['Return'],
        marker_color=['green' if x >= 0 else 'red' for x in monthly_returns_df['Return']],
        name='Monthly Returns'
    ), row=3, col=1)

    # --- Plot 4: Trade PnL Histogram ---
    trade_pnls = [trade['pnl_comm'] for trade in metrics['trade_log'] if 'pnl_comm' in trade]
    if trade_pnls:
        fig.add_trace(go.Histogram(
            x=trade_pnls,
            name='Trade PnL Distribution',\
            marker_color='lightblue',
            opacity=0.7
        ), row=4, col=1)
    else:
        fig.add_annotation(
            x=0.5, y=0.5, text="No trades to display PnL histogram",
            showarrow=False, xref="x4", yref="y4", row=4, col=1
        )


    # === Layout Settings ===
    fig.update_layout(
        title='XAUUSD Strategy Backtest (Multi-Timeframe, Enhanced Logic, Risk Management, Trendlines)',
        xaxis_rangeslider_visible=False,
        legend_title='Legend',
        height=1200, # Increased height for more subplots
        hovermode="x unified"
    )

    # Update axes titles
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Portfolio Value ($)", row=2, col=1)
    fig.update_yaxes(title_text="Return (%)", row=3, col=1, tickformat=".0%")
    fig.update_yaxes(title_text="Frequency", row=4, col=1)
    fig.update_xaxes(title_text="Date", row=4, col=1) # Only show date on the last subplot

    # Hide rangeslider for all subplots
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
    fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
    fig.update_xaxes(rangeslider_visible=False, row=3, col=1)
    fig.update_xaxes(rangeslider_visible=False, row=4, col=1)

    html_file_path = "xauusd_advanced_backtest.html"
    fig.write_html(html_file_path)
    print(f"✅ Visualization saved to '{html_file_path}'")


In [27]:

# --- Robustness & Testing ---
def run_train_test_split(data_15m, data_daily, params):
    """Runs a backtest with a simple train-test split."""
    split_point_15m = int(len(data_15m) * (1 - params['test_split_ratio']))
    split_point_daily = int(len(data_daily) * (1 - params['test_split_ratio']))

    train_data_15m = data_15m.iloc[:split_point_15m]
    test_data_15m = data_15m.iloc[split_point_15m:]

    train_data_daily = data_daily.iloc[:split_point_daily]
    test_data_daily = data_daily.iloc[split_point_daily:]

    print(f"\n--- Running Backtest on Training Data ({train_data_15m.index.min().date()} to {train_data_15m.index.max().date()}) ---")
    train_metrics, _ = run_backtest(train_data_15m, train_data_daily, params, is_optimization=True)
    print(f"Training Net Profit: {train_metrics['net_profit_pct']:.2f}%")
    print(f"Training Max Drawdown: {train_metrics['max_drawdown']:.2f}%")

    print(f"\n--- Running Backtest on Testing Data ({test_data_15m.index.min().date()} to {test_data_15m.index.max().date()}) ---")
    test_metrics, _ = run_backtest(test_data_15m, test_data_daily, params, is_optimization=True)
    print(f"Testing Net Profit: {test_metrics['net_profit_pct']:.2f}%")
    print(f"Testing Max Drawdown: {test_metrics['max_drawdown']:.2f}%")

    return train_metrics, test_metrics

def run_parameter_optimization(data_15m, data_daily, param_ranges):
    """Runs parameter optimization for the strategy."""
    print("\n--- Running Parameter Optimization ---")
    cerebro = bt.Cerebro(optreturn=False) # Return all strategies

    # Add the strategy with optimization parameters
    cerebro.addoptimizer(
        AdvancedStrategy,
        fast_ema=param_ranges.get('fast_ema', [STRATEGY_PARAMS['fast_ema']]),
        slow_ema=param_ranges.get('slow_ema', [STRATEGY_PARAMS['slow_ema']]),
        atr_multiplier_sl=param_ranges.get('atr_multiplier_sl', [STRATEGY_PARAMS['atr_multiplier_sl']]),
        trailing_stop_percent=param_ranges.get('trailing_stop_percent', [STRATEGY_PARAMS['trailing_stop_percent']]),
        risk_per_trade_percent=param_ranges.get('risk_per_trade_percent', [STRATEGY_PARAMS['risk_per_trade_percent']]),
        rsi_oversold=param_ranges.get('rsi_oversold', [STRATEGY_PARAMS['rsi_oversold']]),
        rsi_overbought=param_ranges.get('rsi_overbought', [STRATEGY_PARAMS['rsi_overbought']]),
        adx_min_strength=param_ranges.get('adx_min_strength', [STRATEGY_PARAMS['adx_min_strength']]),
        # Pass other fixed parameters
        atr_period=STRATEGY_PARAMS['atr_period'],
        volume_ma_period=STRATEGY_PARAMS['volume_ma_period'],
        rsi_period=STRATEGY_PARAMS['rsi_period'],
        stoch_rsi_period=STRATEGY_PARAMS['stoch_rsi_period'],
        stoch_rsi_k_period=STRATEGY_PARAMS['stoch_rsi_k_period'],
        stoch_rsi_d_period=STRATEGY_PARAMS['stoch_rsi_d_period'],
        stoch_rsi_oversold=STRATEGY_PARAMS['stoch_rsi_oversold'],
        stoch_rsi_overbought=STRATEGY_PARAMS['stoch_rsi_overbought'],
        macd_fast=STRATEGY_PARAMS['macd_fast'],
        macd_slow=STRATEGY_PARAMS['macd_slow'],
        macd_signal=STRATEGY_PARAMS['macd_signal'],
        adx_period=STRATEGY_PARAMS['adx_period'],
        volatility_threshold_atr_multiplier=STRATEGY_PARAMS['volatility_threshold_atr_multiplier'],
        trading_months=STRATEGY_PARAMS['trading_months'],
        fixed_commission=STRATEGY_PARAMS['fixed_commission'],
        percent_commission=STRATEGY_PARAMS['percent_commission'],
        slippage_percent=STRATEGY_PARAMS['slippage_percent'],
        leverage=STRATEGY_PARAMS['leverage'],
        initial_cash=STRATEGY_PARAMS['initial_cash'],
        swing_lookback_period=STRATEGY_PARAMS['swing_lookback_period'],
        # NEW: Trendline parameters for optimization
        enable_trendline_logic=param_ranges.get('enable_trendline_logic', [STRATEGY_PARAMS['enable_trendline_logic']]),
        trendline_length=param_ranges.get('trendline_length', [STRATEGY_PARAMS['trendline_length']]),
        slope_multiplier=param_ranges.get('slope_multiplier', [STRATEGY_PARAMS['slope_multiplier']]),
        slope_calc_method=param_ranges.get('slope_calc_method', [STRATEGY_PARAMS['slope_calc_method']])
    )

    # Add data feeds
    cerebro.adddata(bt.feeds.PandasData(dataname=data_15m, timeframe=bt.TimeFrame.Minutes, compression=15))
    cerebro.adddata(bt.feeds.PandasData(dataname=data_daily, timeframe=bt.TimeFrame.Days, compression=1))

    # Broker settings (same as run_backtest)
    cerebro.broker.set_cash(STRATEGY_PARAMS['initial_cash'])
    cerebro.broker.setcommission(commission=STRATEGY_PARAMS['percent_commission'],
                                 commtype=bt.CommInfoBase.COMM_PERC,
                                 mult=1.0,
                                 leverage=STRATEGY_PARAMS['leverage'])
    cerebro.broker.set_slippage_perc(STRATEGY_PARAMS['slippage_percent'])
    cerebro.broker.set_fundmode(True, fundstartval=STRATEGY_PARAMS['initial_cash'])

    # Add analyzers for optimization
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
    # Add Sortino ratio for optimization, if available
    try:
        cerebro.addanalyzer(bt.analyzers.SortinoRatio, _name='sortino_ratio')
    except AttributeError:
        print("Warning: SortinoRatio analyzer not found for optimization. Skipping.")
        class DummySortinoOpt:
            def get_analysis(self): return {'sortinoratio': 'N/A'}
        cerebro.addanalyzer(DummySortinoOpt, _name='sortino_ratio')


    # Run optimization
    optimized_runs = cerebro.run()

    best_params = None
    best_profit = -float('inf')
    results_summary = []

    for run in optimized_runs:
        for strategy in run:
            # Extract parameters
            params = strategy.p._getkwargs()

            # Extract metrics
            net_profit_pct = (strategy.broker.getvalue() - STRATEGY_PARAMS['initial_cash']) / STRATEGY_PARAMS['initial_cash'] * 100
            max_drawdown = strategy.analyzers.drawdown.get_analysis().get('max', {}).get('drawdown', 0.0)
            sharpe_ratio = strategy.analyzers.sharpe_ratio.get_analysis().get('sharperatio', 'N/A')
            sortino_ratio = strategy.analyzers.sortino_ratio.get_analysis().get('sortinoratio', 'N/A')

            results_summary.append({
                'params': params,
                'net_profit_pct': net_profit_pct,
                'max_drawdown': max_drawdown,
                'sharpe_ratio': sharpe_ratio,
                'sortino_ratio': sortino_ratio
            })

            if net_profit_pct > best_profit:
                best_profit = net_profit_pct
                best_params = params

    print("\n--- Optimization Results Summary ---")
    for res in results_summary:
        print(f"Params: {res['params']} | Profit: {res['net_profit_pct']:.2f}% | Max Drawdown: {res['max_drawdown']:.2f}% | Sharpe: {res['sharpe_ratio']} | Sortino: {res['sortino_ratio']}")

    if best_params:
        print(f"\nBest Parameters (by Net Profit): {best_params}")
        print(f"Best Profit: {best_profit:.2f}%")
    else:
        print("No optimal parameters found.")

    return best_params, results_summary

# --- Main Execution ---
if __name__ == '__main__':
    print("Starting XAUUSD Strategy Backtest and Optimization...")

    # --- 1. Run Main Backtest with Default Parameters ---
    print("\n--- Running Main Backtest ---")
    main_metrics, main_strat = run_backtest(df_15m, df_daily, STRATEGY_PARAMS)
    print(f"Main Backtest Final Value: ₹{main_metrics['final_value']:.2f}")
    print(f"Main Backtest Net Profit: ₹{main_metrics['net_profit']:.2f} ({main_metrics['net_profit_pct']:.2f}%)\n")
    print(f"Main Backtest Max Drawdown: {main_metrics['max_drawdown']:.2f}%")
    print(f"Main Backtest Sharpe Ratio: {main_metrics['sharpe_ratio']}")
    print(f"Main Backtest Sortino Ratio: {main_metrics['sortino_ratio']}")


    # --- 2. Run Train-Test Split (Uncomment to enable) ---
    # print("\n--- Running Train-Test Split ---")
    # train_metrics, test_metrics = run_train_test_split(df_15m, df_daily, STRATEGY_PARAMS)
    # print(f"\nTrain-Test Split Complete. Training Profit: {train_metrics['net_profit_pct']:.2f}%, Testing Profit: {test_metrics['net_profit_pct']:.2f}%")

    # --- 3. Run Parameter Optimization (Example ranges - Uncomment to enable) ---
    # Be cautious with wide ranges as it can take a very long time
    # param_optimization_ranges = {
    #     'fast_ema': [20, 50, 80],
    #     'slow_ema': [50, 100, 150],
    #     'atr_multiplier_sl': [2.0, 2.5, 3.0],
    #     'trailing_stop_percent': [0.01, 0.015, 0.02],
    #     'risk_per_trade_percent': [0.005, 0.01, 0.015],
    #     'adx_min_strength': [20, 25, 30],
    #     # Add new trendline parameters to optimization ranges if desired
    #     # 'trendline_length': [10, 14, 20],
    #     # 'slope_multiplier': [0.5, 1.0, 1.5],
    #     # 'slope_calc_method': ['Atr', 'Stdev'] # Optimize between calculation methods
    # }
    # best_opt_params, optimization_summary = run_parameter_optimization(df_15m, df_daily, param_optimization_ranges)
    # if best_opt_params:
    #     print(f"\nOptimization Best Parameters: {best_opt_params}")

    print("\nBacktest and Optimization Complete.")

Starting XAUUSD Strategy Backtest and Optimization...

--- Running Main Backtest ---
2004-11-01T23:00:00 - TRENDLINE: Upward Breakout Detected at 2004-11-01T23:00:00 - Price: 426.80
2004-11-02T23:00:00 - TRENDLINE: Upward Breakout Detected at 2004-11-02T23:00:00 - Price: 421.30
2004-11-03T12:30:00 - TRENDLINE: Upward Breakout Detected at 2004-11-03T12:30:00 - Price: 422.30
2004-11-03T20:15:00 - TRENDLINE: Upward Breakout Detected at 2004-11-03T20:15:00 - Price: 425.30
2004-11-04T01:00:00 - TRENDLINE: Upward Breakout Detected at 2004-11-04T01:00:00 - Price: 426.80
2004-11-04T07:15:00 - TRENDLINE: Upward Breakout Detected at 2004-11-04T07:15:00 - Price: 427.20
2004-11-04T14:00:00 - TRENDLINE: Upward Breakout Detected at 2004-11-04T14:00:00 - Price: 427.80
2004-11-05T12:45:00 - TRENDLINE: Upward Breakout Detected at 2004-11-05T12:45:00 - Price: 429.80
2004-11-05T19:00:00 - TRENDLINE: Upward Breakout Detected at 2004-11-05T19:00:00 - Price: 433.20
2004-11-08T01:45:00 - TRENDLINE: Upward Br